<a href="https://colab.research.google.com/github/Iddo77/YOLO_drone_detection/blob/main/YOLO_drones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install scikit-learn opencv-python-headless pillow numpy

                                              0.0/38.1 MB ? eta -:--:--
                                              0.3/38.1 MB 8.9 MB/s eta 0:00:05
                                              0.7/38.1 MB 9.2 MB/s eta 0:00:05
     -                                        1.0/38.1 MB 8.3 MB/s eta 0:00:05
     -                                        1.4/38.1 MB 8.2 MB/s eta 0:00:05
     -                                        1.6/38.1 MB 7.2 MB/s eta 0:00:06
     --                                       1.9/38.1 MB 7.1 MB/s eta 0:00:06
     --                                       2.2/38.1 MB 7.2 MB/s eta 0:00:06
     --                                       2.7/38.1 MB 7.4 MB/s eta 0:00:05
     ---                                      3.0/38.1 MB 7.4 MB/s eta 0:00:05
     ---                                      3.3/38.1 MB 7.3 MB/s eta 0:00:05
     ---                                      3.7/38.1 MB 7.5 MB/s eta 0:00:05
     ----                                     4.1/38.1 MB 7

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\OU Master AI\\Research methods\\Project\\yolo-drones-env\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [2]:
import os
import random
import shutil
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image, ImageEnhance
from typing import List, Tuple

In [3]:
# constants
NUM_TRAIN = 500
RANDOM_STATE = 36
# replace with correct root of project 
PROJECT_ROOT = r"D:\OU Master AI\Research methods\Project"

In [4]:
# get the path to our own project
project_path = os.path.join(PROJECT_ROOT, 'YOLO_drone_detection')
print(project_path)
# get a the path to the dataset folder
dataset_path = os.path.join(project_path, "dataset")
print(dataset_path)
# get the path to the folder with the YOLO v7 code
yolo_path = os.path.join(PROJECT_ROOT, "yolov7")
print(yolo_path)

D:\OU Master AI\Research methods\Project\YOLO_drone_detection
D:\OU Master AI\Research methods\Project\YOLO_drone_detection\dataset
D:\OU Master AI\Research methods\Project\yolov7


In [5]:
def create_train_val_test_txt_files():
    # get all image files and shuffle them
    all_images = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]
    random.shuffle(all_images)

    # Split the dataset into training, validation, and test sets
    train_images = all_images[:NUM_TRAIN]
    test_val_images = all_images[NUM_TRAIN:]

    # split the test_val set into validation and test sets
    val_images, test_images = train_test_split(test_val_images, 
                                               train_size=0.5, 
                                               test_size=0.5, 
                                               random_state=RANDOM_STATE)

    # Write the paths to the train, val, and test text files
    with open(os.path.join(project_path, 'train.txt'), 'w') as f:
        for img in train_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'val.txt'), 'w') as f:
        for img in val_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'test.txt'), 'w') as f:
        for img in test_images:
            f.write(os.path.join(dataset_path, img) + '\n')

In [6]:
def create_custom_yaml_file():
    """
    Creates a custom YAML file for YOLO that specifies the paths to the training, 
    validation, and test sets.
    """
    yaml_text = f"""train: '{os.path.join(project_path, "train.txt")}'
val: '{os.path.join(project_path, "val.txt")}'
test: '{os.path.join(project_path, "test.txt")}'
nc: 1
names: ['drone']
"""
    with open(os.path.join(project_path, 'custom.yaml'), 'w') as f:
        f.write(yaml_text)


In [6]:
create_train_val_test_txt_files()
create_custom_yaml_file()

In [13]:
# finetune the base model using the custom.yaml file created above
os.chdir(yolo_path)
!python train.py --workers 8 --device 0 --batch-size 8 --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.30, Best Possible Recall (BPR) = 1.0000
                 all         529         579     0.00367       0.722      0.0083     0.00105
                 all         529         579     0.00923       0.617     0.00697    0.000859
                 all         529         579     0.00365      0.0759     0.00746    0.000991
                 all         529         579     0.00898      0.0345     0.00937     0.00264
                 all         529         579     0.00684       0.325     0.00455    0.000686
                 all         529         579       0.011      0.0691     0.00541    0.000792
                 all         529         579      0.0265      0.0725     0.00871     0.00129
                 all         529         579     0.00315       0.472     0.00245    0.000368
                 all         529         579     0.00507        0.25     0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# finetune with 500 epochs instead of 300
os.chdir(yolo_path)
!python train.py --workers 8 --device 0 --batch-size 8 --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml --epochs 500

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.24, Best Possible Recall (BPR) = 1.0000
                 all         429         471     0.00982       0.603     0.00693    0.000857
                 all         429         471     0.00351       0.747     0.00375    0.000499
                 all         429         471      0.0194      0.0276      0.0047    0.000733
                 all         429         471      0.0239      0.0234     0.00554    0.000814
                 all         429         471      0.0635       0.034     0.00888     0.00151
                 all         429         471      0.0118       0.151      0.0101     0.00188
                 all         429         471     0.00505       0.168     0.00341    0.000528
                 all         429         471      0.0841      0.0764      0.0264     0.00564
                 all         429         471      0.0285       0.087     0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [421]:
# get a the path to the augmented dataset folder
augmented_dataset_path = os.path.join(project_path, "augmented-dataset")
os.makedirs(augmented_dataset_path, exist_ok=True)
print(augmented_dataset_path)

D:\OU Master AI\Research methods\Project\YOLO_drone_detection\augmented-dataset


In [422]:
def create_augmented_dataset(n=50):
    
    # Read train.txt to get the list of image paths
    with open(os.path.join(project_path, 'train.txt'), 'r') as f:
        image_files = f.read().splitlines()
    

    # Randomly select n images files
    random_image_files = random.sample(image_files, n)

    for file_path in random_image_files:
        filename = os.path.basename(file_path)
        destination_path = os.path.join(augmented_dataset_path, filename)
        
        # Copy the image file
        shutil.copy(file_path, destination_path)
        
        # Get the corresponding bounding box file path
        base = os.path.splitext(file_path)[0]
        bbox_file_path = f"{base}.txt"
        
        # Copy the bounding box file
        shutil.copy(bbox_file_path, augmented_dataset_path)

In [423]:
create_augmented_dataset()

In [424]:
def jitter_image_and_copy_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function applies color jitter to the image, saves the new image, and copies the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Apply color jitter
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    random_bright = .25 + np.random.uniform()
    image[:, :, 2] = image[:, :, 2] * random_bright
    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_jittered.jpg")
    cv2.imwrite(new_image_path, image)

    # Copy the bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_jittered.txt")
    shutil.copy(bbox_path, new_bbox_path)


In [425]:
def alter_brightness_and_copy_bbox(image: np.ndarray, bbox_path: str, image_name: str, factor: float) -> None:
    """
    This function alters brightness of the image, saves the new image, and copies the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
        factor: brightness control (1.0 gives original image)
    """
    
    # Convert to PIL image and  alter the brightness
    pil_img = Image.fromarray(image)
    enhancer = ImageEnhance.Brightness(pil_img)
    enhanced_im = enhancer.enhance(factor)
    
     # Convert back to numpy and save the new image
    image = np.asarray(enhanced_im)
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_brightness_altered.jpg")
    cv2.imwrite(new_image_path, image)
    
    # Copy the bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_brightness_altered.txt")
    shutil.copy(bbox_path, new_bbox_path)


In [426]:
def alter_contrast_and_copy_bbox(image: np.ndarray, bbox_path: str, image_name: str, factor: float) -> None:
    """
    This function alters contrast of the image, saves the new image, and copies the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
        factor: contrast control (1.0 gives original image)
    """
    # Convert to PIL image and alter the contrast
    pil_img = Image.fromarray(image)
    enhancer = ImageEnhance.Contrast(pil_img)
    enhanced_im = enhancer.enhance(factor)

    # Convert back to numpy and save the new image
    image = np.asarray(enhanced_im)
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_contrast_altered.jpg")
    cv2.imwrite(new_image_path, image)

    # Copy the bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_contrast_altered.txt")
    shutil.copy(bbox_path, new_bbox_path)

In [427]:
def flip_image_horizontally_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function flips an image horizontally, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Flip the image horizontally
    image = cv2.flip(image, 1)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_hori.jpg")
    cv2.imwrite(new_image_path, image)

    # Update the bounding box
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    bbox_data[:, 1] = 1 - bbox_data[:, 1]  # flip x-center

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_hori.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])


In [428]:
def flip_image_vertically_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function flips an image vertically, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Flip the image vertically
    image = cv2.flip(image, 0)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_vert.jpg")
    cv2.imwrite(new_image_path, image)

    # Update the bounding box
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    bbox_data[:, 2] = 1 - bbox_data[:, 2]  # flip y-center

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_vert.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [429]:
def rotate_image_90_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str, clockwise: bool) -> None:
    """
    This function rotates an image by 90 degrees, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
        clockwise: If True, rotates the image clockwise. If False, rotates counterclockwise.
    """
    # Get suffix for filenames and rotation factor for OpenCV
    if clockwise:
        suffix = 'clockwise'
        rotation_factor = cv2.ROTATE_90_CLOCKWISE
    else:
        suffix = 'counterclockwise'
        rotation_factor = cv2.ROTATE_90_COUNTERCLOCKWISE

    # Rotate the image
    image = cv2.rotate(image, rotation_factor)
    
    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_rotated_{suffix}.jpg")
    cv2.imwrite(new_image_path, image)

    # Update the bounding box
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    if clockwise:
        bbox_data[:, [1, 2]] = bbox_data[:, [2, 1]]
        bbox_data[:, [3, 4]] = bbox_data[:, [4, 3]]
        bbox_data[:, 1] = 1 - bbox_data[:, 1]
    else:
        bbox_data[:, [1, 2]] = 1 - bbox_data[:, [2, 1]]
        bbox_data[:, [3, 4]] = bbox_data[:, [4, 3]]
        bbox_data[:, 1] = 1 - bbox_data[:, 1]

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_rotated_{suffix}.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [430]:
def conditional_zoom_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function conditionally zooms an image, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Load the bounding box data
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    
    # Calculate the area of the bounding box and the image
    bbox_area = ((bbox_data[:, 3] - bbox_data[:, 1]) * (bbox_data[:, 4] - bbox_data[:, 2])).sum()

    # Choose zoom direction based on the bounding box area
    zoom_factor = 1.3 if bbox_area > 0.5 else 0.7

    # Resize the image
    image = cv2.resize(image, None, fx=zoom_factor, fy=zoom_factor, interpolation=cv2.INTER_LINEAR)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_zoomed.jpg")
    cv2.imwrite(new_image_path, image)

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_zoomed.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [431]:
def conditional_translate_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function conditionally translates an image, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Load the bounding box data
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    
    # Calculate the center of the bounding box
    bbox_center = bbox_data[:, 1:3].mean(axis=0)

    # Define translation distances (10% of image width/height)
    translate_dist = np.array([0.1, 0])  # translate horizontally

    # Choose translation direction based on the bounding box center
    translate_dist *= -1 if bbox_center[0] < 0.5 else 1  # left if center is on the left half, else right

    # Create the translation matrix
    M = np.float32([[1, 0, translate_dist[0]*image.shape[1]], [0, 1, translate_dist[1]*image.shape[0]]])

    # Translate the image
    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Update the bounding box center
    bbox_data[:, 1:3] += translate_dist

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_translated.jpg")
    cv2.imwrite(new_image_path, image)

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_translated.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [432]:
def augment_image(image_path: str) -> None:
    """
    This function applies various augmentations to an image.
    
    Args:
        image_path: Full path to the image file.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Extract the image name (without extension) and bounding box path
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    bbox_path = os.path.join(os.path.dirname(image_path), f"{image_name}.txt")

    # Apply various transformations and save the results
    jitter_image_and_copy_bbox(image, bbox_path, image_name)
    alter_brightness_and_copy_bbox(image, bbox_path, image_name, 1.5)
    alter_contrast_and_copy_bbox(image, bbox_path, image_name, 1.5)
    flip_image_horizontally_and_update_bbox(image, bbox_path, image_name)
    flip_image_vertically_and_update_bbox(image, bbox_path, image_name)
    rotate_image_90_and_update_bbox(image, bbox_path, image_name, clockwise=True)
    rotate_image_90_and_update_bbox(image, bbox_path, image_name, clockwise=False)
    conditional_zoom_and_update_bbox(image, bbox_path, image_name)
    conditional_translate_and_update_bbox(image, bbox_path, image_name)

In [433]:
def augment_all_images() -> None:
    """
    This function finds all images in the augmented_dataset_path directory and applies various augmentations to each image.
    """
    # Find all .jpg image files in the directory
    image_paths = glob.glob(os.path.join(augmented_dataset_path, '*.jpg'))

    # Apply augmentations to each image
    for image_path in image_paths:
        augment_image(image_path)

In [434]:
augment_all_images()

In [435]:
def draw_bounding_boxes(image_path: str, bbox_path: str, output_dir: str) -> None:
    """
    This function draws bounding boxes on an image and saves the image.

    Args:
        image_path: Full path to the image file.
        bbox_path: Full path to the corresponding bounding box file.
        output_dir: Directory to save the images with bounding boxes.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Load the bounding boxes
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)

    # Draw each bounding box on the image
    for bbox in bbox_data:
        # Convert bbox from relative to absolute coordinates
        abs_bbox = np.zeros_like(bbox)
        abs_bbox[0] = bbox[0]
        abs_bbox[1] = (bbox[1] - bbox[3] / 2) * image.shape[1]  # x_min
        abs_bbox[2] = (bbox[2] - bbox[4] / 2) * image.shape[0]  # y_min
        abs_bbox[3] = (bbox[1] + bbox[3] / 2) * image.shape[1]  # x_max
        abs_bbox[4] = (bbox[2] + bbox[4] / 2) * image.shape[0]  # y_max

        abs_bbox = abs_bbox.astype(int)

        # Draw the bounding box
        cv2.rectangle(image, (abs_bbox[1], abs_bbox[2]), (abs_bbox[3], abs_bbox[4]), (0, 255, 0), 2)

    # Save the image
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, image)

In [436]:
def draw_bboxes_for_all_images_in_folder(input_dir: str, output_dir: str) -> None:
    """
    This function finds all images in the input directory and draws bounding boxes on each image.

    Args:
        input_dir: Directory containing the images and bounding box files.
        output_dir: Directory to save the images with bounding boxes.
    """
    # Find all .jpg image files in the directory
    image_paths = glob.glob(os.path.join(input_dir, '*.jpg'))

    # For each image, draw bounding boxes and save the image
    for image_path in image_paths:
        # Derive the bounding box file path from the image file path
        bbox_path = image_path.rsplit('.', 1)[0] + '.txt'
        draw_bounding_boxes(image_path, bbox_path, output_dir)

In [437]:
output_dir = os.path.join(os.path.dirname(augmented_dataset_path), "augmented-dataset-validation")

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.mkdir(output_dir)

draw_bboxes_for_all_images_in_folder(augmented_dataset_path, output_dir)